In [ ]:
import httpx  # install via pip install httpx
import io
import pandas as pd

BASE_URL = "http://localhost:25503/v3"  # all endpoints use this URL base

# set params for the first request
params = {
    "symbol": "SPX",
    "start_date": "20220103",
    "end_date": "20220103",
    "interval": "1m",
}

url = BASE_URL + "/index/history/ohlc"

df = pd.DataFrame()  # collect page DataFrames here

resp = httpx.get(url, params=params, timeout=60)
resp.raise_for_status()  # make sure the request worked

text = resp.text
if text.strip() != "":
    # pandas will parse the header row as column names
    df = pd.read_csv(io.StringIO(text))


df.head(10)

In [11]:
import thetadata as td
import pandas as pd

df = td.option_list_expirations("SPXW")



In [12]:
df=df[(df['expiration'] >= "2017-01-01") & (df['expiration'] <= "2025-12-31")]

df.tail(10)

,symbol,expiration
1992,SPXW,2025-11-13
1993,SPXW,2025-11-14
1994,SPXW,2025-11-17
1995,SPXW,2025-11-18
1996,SPXW,2025-11-21
1997,SPXW,2025-11-28
1998,SPXW,2025-12-05
1999,SPXW,2025-12-12
2000,SPXW,2025-12-19
2001,SPXW,2025-12-31


In [6]:
import ifera
import torch
import re

with open('data/results/option_alpha/filters/SPX-ORB-L-ADX_14.txt', 'r') as f:
    html = f.read()

dff = ifera.parse_filter_log(html)

def parse_range_width(description):
    match = re.search(r'Opening Range: ([\d,]+\.\d+) - ([\d,]+\.\d+)', description)
    if match:
        min_val = float(match.group(1).replace(',', ''))
        max_val = float(match.group(2).replace(',', ''))
        return max_val - min_val
    else:
        return None

# dff['range_width'] = dff['description'].apply(parse_range_width)
dff['ADX_14'] = dff['description'].astype(float)

In [8]:
dff.head(10)

,date,filter_type,description,ADX_14
0,2022-01-06,ADX(14),17.5545,17.5545
1,2022-01-10,ADX(14),15.493,15.4930
2,2022-01-11,ADX(14),16.7245,16.7245
3,2022-01-14,ADX(14),14.6198,14.6198
4,2022-01-20,ADX(14),18.1388,18.1388
5,2022-01-21,ADX(14),20.1889,20.1889
6,2022-01-24,ADX(14),23.0378,23.0378
7,2022-01-25,ADX(14),26.2493,26.2493
8,2022-01-26,ADX(14),28.177,28.1770
9,2022-01-28,ADX(14),31.0415,31.0415


In [ ]:
import ifera
dff = ifera.get_filters("SPX-ORB-L")

dff.head()

,date,range_width
0,2022-01-03,37.69
1,2022-01-04,14.11
2,2022-01-05,19.28
3,2022-01-06,49.48
4,2022-01-07,23.75


In [ ]:
import ifera
import torch
import time

trade_grid = """
<bd class="dim-scroller"><row data-key="p1762062476073700000051167"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 21, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:51am → 3:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,850</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$730</span></div></bd></row><row data-key="p1762062476073700000051168"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 24, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:32pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,987</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$6</span></div></bd></row><row data-key="p1762062476073700000051169"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 26, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:03pm → 3:09pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,675</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,510</span></div></bd></row><row data-key="p1762062476073700000051170"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 28, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:00am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$265</span></div></bd></row><row data-key="p1762062476073700000051171"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 31, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:51am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,920</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$80</span></div></bd></row><row data-key="p1762062476073700000051172"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:48pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,830</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$170</span></div></bd></row><row data-key="p1762062476073700000051173"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 4, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:01pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p1762062476073700000051174"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 7, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,673</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$445</span></div></bd></row><row data-key="p1762062476073700000051175"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 9, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$197</span></div></bd></row><row data-key="p1762062476073700000051176"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 14, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:32pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,772</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$166</span></div></bd></row><row data-key="p1762062476073700000051177"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 16, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:22pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,882</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$118</span></div></bd></row><row data-key="p1762062476073700000051178"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 22, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 2:11pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,715</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,310</span></div></bd></row><row data-key="p1762062476073700000051179"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:43am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,730</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$270</span></div></bd></row><row data-key="p1762062476073700000051180"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 28, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,560</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$400</span></div></bd></row><row data-key="p1762062476073700000051181"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,565</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$435</span></div></bd></row><row data-key="p1762062476073700000051182"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 9, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,720</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$280</span></div></bd></row><row data-key="p1762062476073700000051183"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 14, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:52am → 1:34pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,805</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$940</span></div></bd></row><row data-key="p1762062476073700000051184"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 18, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,849</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$151</span></div></bd></row><row data-key="p1762062476073700000051185"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 23, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,710</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$207</span></div></bd></row><row data-key="p1762062476073700000051186"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,745</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$198</span></div></bd></row><row data-key="p1762062476073700000051187"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 28, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:45pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,935</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$65</span></div></bd></row><row data-key="p1762062476073700000051188"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 31, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,663</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,453</span></div></bd></row><row data-key="p1762062476073700000051189"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 4, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$157</span></div></bd></row><row data-key="p1762062476073700000051190"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 6, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:03pm → 2:13pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,913</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$450</span></div></bd></row><row data-key="p1762062476073700000051191"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 8, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:52am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,765</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$225</span></div></bd></row><row data-key="p1762062476073700000051192"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 13, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:04am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,792</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$208</span></div></bd></row><row data-key="p1762062476073700000051193"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 18, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:56pm → 3:31pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,907</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$449</span></div></bd></row><row data-key="p1762062476073700000051194"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$265</span></div></bd></row><row data-key="p1762062476073700000051195"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 27, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:28pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$5</span></div></bd></row><row data-key="p1762062476073700000051196"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 1:47pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,003</span></div></bd></row><row data-key="p1762062476073700000051197"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 3, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,715</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$268</span></div></bd></row><row data-key="p1762062476073700000051198"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 4, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:40pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,495</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$505</span></div></bd></row><row data-key="p1762062476073700000051199"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 6, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:47am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,710</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$290</span></div></bd></row><row data-key="p1762062476073700000051200"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 11, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 1:47pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,780</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,045</span></div></bd></row><row data-key="p1762062476073700000051201"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 13, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:28am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,870</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$130</span></div></bd></row><row data-key="p1762062476073700000051202"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 16, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:03pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,800</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$190</span></div></bd></row><row data-key="p1762062476073700000051203"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 17, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:44pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,705</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$295</span></div></bd></row><row data-key="p1762062476073700000051204"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 23, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,720</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$280</span></div></bd></row><row data-key="p1762062476073700000051205"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,700</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$300</span></div></bd></row><row data-key="p1762062476073700000051206"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 26, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:00am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$70</span></div></bd></row><row data-key="p1762062476073700000051207"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 27, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:43am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,960</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$40</span></div></bd></row><row data-key="p1762062476073700000051208"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 31, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:57am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$108</span></div></bd></row><row data-key="p1762062476073700000051209"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:03am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,802</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$198</span></div></bd></row><row data-key="p1762062476073700000051210"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 7, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,780</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$220</span></div></bd></row><row data-key="p1762062476073700000051211"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 21, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,905</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$95</span></div></bd></row><row data-key="p1762062476073700000051212"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 22, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$70</span></div></bd></row><row data-key="p1762062476073700000051213"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 23, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,718</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$282</span></div></bd></row><row data-key="p1762062476073700000051214"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 24, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:06am → 3:54pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,963</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$30</span></div></bd></row><row data-key="p1762062476073700000051215"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 30, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:27am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476073700000051216"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 1, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:47pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,993</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$7</span></div></bd></row><row data-key="p1762062476073700000051217"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 5, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:44pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,972</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$28</span></div></bd></row><row data-key="p1762062476073700000051218"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 6, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:34pm → 3:52pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,893</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$97</span></div></bd></row><row data-key="p1762062476073700000051219"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 7, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:53am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,857</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$143</span></div></bd></row><row data-key="p1762062476073700000051220"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 8, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,865</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p1762062476073700000051221"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 13, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$265</span></div></bd></row><row data-key="p1762062476073700000051222"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 14, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:43am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,915</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$85</span></div></bd></row><row data-key="p1762062476073700000051223"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 15, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,820</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$180</span></div></bd></row><row data-key="p1762062476073700000051224"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 19, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476073700000051225"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 20, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$212</span></div></bd></row><row data-key="p1762062476073700000051226"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 21, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$70</span></div></bd></row><row data-key="p1762062476073700000051227"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:36am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,804</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$182</span></div></bd></row><row data-key="p1762062476073700000051228"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 27, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:56am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,435</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$565</span></div></bd></row><row data-key="p1762062476073700000051229"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 28, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:24am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,945</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$55</span></div></bd></row><row data-key="p1762062476073700000051230"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 29, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:54pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,993</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$7</span></div></bd></row><row data-key="p1762062476073700000051231"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 1, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:51am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,878</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$122</span></div></bd></row><row data-key="p1762062476073700000051232"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:44am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,770</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$230</span></div></bd></row><row data-key="p1762062476073700000051233"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 3, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:13am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p1762062476073700000051234"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 10, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:01am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,815</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$185</span></div></bd></row><row data-key="p1762062476073799999951235"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 12, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:03am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,840</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$160</span></div></bd></row><row data-key="p1762062476073799999951236"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 15, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:45am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,898</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$102</span></div></bd></row><row data-key="p1762062476073799999951237"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 16, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:18am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,723</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$277</span></div></bd></row><row data-key="p1762062476073799999951238"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 17, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:03pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,768</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$192</span></div></bd></row><row data-key="p1762062476073799999951239"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 18, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:49am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,730</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$270</span></div></bd></row><row data-key="p1762062476073799999951240"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 24, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:15am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,904</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$96</span></div></bd></row><row data-key="p1762062476073799999951241"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$90</span></div></bd></row><row data-key="p1762062476073799999951242"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 29, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:49pm → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,873</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p1762062476073799999951243"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 1, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:23pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,952</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$48</span></div></bd></row><row data-key="p1762062476073799999951244"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 1:09pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,795</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$960</span></div></bd></row><row data-key="p1762062476073799999951245"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 6, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:09am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,890</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$100</span></div></bd></row><row data-key="p1762062476073799999951246"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 7, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,680</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$320</span></div></bd></row><row data-key="p1762062476073799999951247"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 8, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,868</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$132</span></div></bd></row><row data-key="p1762062476073799999951248"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 9, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,795</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$205</span></div></bd></row><row data-key="p1762062476073799999951249"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 14, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:55am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,777</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$136</span></div></bd></row><row data-key="p1762062476073799999951250"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 19, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,755</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$245</span></div></bd></row><row data-key="p1762062476073799999951251"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 20, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:06pm → 1:56pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,805</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$935</span></div></bd></row><row data-key="p1762062476073799999951252"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 21, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,235</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,220</span></div></bd></row><row data-key="p1762062476073799999951253"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 28, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,700</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$300</span></div></bd></row><row data-key="p1762062476073799999951254"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 30, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,723</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$335</span></div></bd></row><row data-key="p1762062476073799999951255"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 3, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,907</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$93</span></div></bd></row><row data-key="p1762062476073799999951256"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 4, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:59am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,942</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$58</span></div></bd></row><row data-key="p1762062476073799999951257"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 5, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:19pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,902</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$98</span></div></bd></row><row data-key="p1762062476073799999951258"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 11, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:04am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,833</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$157</span></div></bd></row><row data-key="p1762062476073799999951259"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 12, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:08pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,787</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$199</span></div></bd></row><row data-key="p1762062476073799999951260"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 13, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,832</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$168</span></div></bd></row><row data-key="p1762062476073799999951261"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 21, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:02pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,938</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$62</span></div></bd></row><row data-key="p1762062476073799999951262"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 24, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:43pm → 2:44pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$8</span></div></bd></row><row data-key="p1762062476073799999951263"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 25, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476073799999951264"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 26, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,844</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$144</span></div></bd></row><row data-key="p1762062476073799999951265"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 27, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,832</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$141</span></div></bd></row><row data-key="p1762062476073799999951266"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 28, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,898</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$102</span></div></bd></row><row data-key="p1762062476073799999951267"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 31, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:25am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,717</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$269</span></div></bd></row><row data-key="p1762062476073799999951268"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 3, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,847</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$153</span></div></bd></row><row data-key="p1762062476073799999951269"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 4, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 11:41am</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$475</span></div></bd></row><row data-key="p1762062476073799999951270"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 7, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:06pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,970</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$30</span></div></bd></row><row data-key="p1762062476073799999951271"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 8, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 2:27pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,838</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$748</span></div></bd></row><row data-key="p1762062476073799999951272"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 10, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:59am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,890</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$110</span></div></bd></row><row data-key="p1762062476073799999951273"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 11, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:25pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,975</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$25</span></div></bd></row><row data-key="p1762062476073799999951274"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 14, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:37pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,913</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$25</span></div></bd></row><row data-key="p1762062476073799999951275"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 17, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:56am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,785</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$215</span></div></bd></row><row data-key="p1762062476073799999951276"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 22, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,745</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$255</span></div></bd></row><row data-key="p1762062476073799999951277"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 23, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,852</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$148</span></div></bd></row><row data-key="p1762062476073799999951278"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 29, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 11:20am</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$928</span></div></bd></row><row data-key="p1762062476073799999951279"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 30, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:31pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,540</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$460</span></div></bd></row><row data-key="p1762062476073799999951280"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 2, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,765</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$235</span></div></bd></row><row data-key="p1762062476073799999951281"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 7, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:53am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,810</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476073799999951282"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 8, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,887</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$103</span></div></bd></row><row data-key="p1762062476073799999951283"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 9, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:00am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,870</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$74</span></div></bd></row><row data-key="p1762062476073799999951284"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 12, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:05am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,725</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$275</span></div></bd></row><row data-key="p1762062476073799999951285"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 14, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:45am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,345</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$875</span></div></bd></row><row data-key="p1762062476073799999951286"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 20, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,893</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$107</span></div></bd></row><row data-key="p1762062476073799999951287"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 21, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,894</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$106</span></div></bd></row><row data-key="p1762062476073899999981288"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 23, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p1762062476073899999981289"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 27, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:47am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,860</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$140</span></div></bd></row><row data-key="p1762062476073899999981290"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 29, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,920</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$80</span></div></bd></row><row data-key="p1762062476073899999981291"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 30, 2022</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:47pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,995</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$5</span></div></bd></row><row data-key="p1762062476073899999981292"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 4, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,867</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$133</span></div></bd></row><row data-key="p1762062476073899999981293"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 6, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,922</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$78</span></div></bd></row><row data-key="p1762062476073899999981294"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 9, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 3:15pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,870</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$742</span></div></bd></row><row data-key="p1762062476073899999981295"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 10, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:13pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,980</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$20</span></div></bd></row><row data-key="p1762062476073899999981296"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 11, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:11pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,757</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$243</span></div></bd></row><row data-key="p1762062476073899999981297"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 12, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,880</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$120</span></div></bd></row><row data-key="p1762062476073899999981298"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,865</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p1762062476073899999981299"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 19, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:41pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,901</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$89</span></div></bd></row><row data-key="p1762062476073899999981300"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 20, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p1762062476073899999981301"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 23, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,902</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$98</span></div></bd></row><row data-key="p1762062476073899999981302"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 24, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:55am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,839</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$161</span></div></bd></row><row data-key="p1762062476073899999981303"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 25, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:52am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,858</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$142</span></div></bd></row><row data-key="p1762062476073899999981304"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 27, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:39pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,960</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$40</span></div></bd></row><row data-key="p1762062476073899999981305"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 31, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,740</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$260</span></div></bd></row><row data-key="p1762062476073899999981306"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 1, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:40pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,585</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$415</span></div></bd></row><row data-key="p1762062476073899999981307"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 2, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,792</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$194</span></div></bd></row><row data-key="p1762062476073899999981308"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 3, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$102</span></div></bd></row><row data-key="p1762062476073899999981309"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 6, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:25am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,802</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$181</span></div></bd></row><row data-key="p1762062476073899999981310"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 7, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:09am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,400</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$600</span></div></bd></row><row data-key="p1762062476073899999981311"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 10, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:21pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,892</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$108</span></div></bd></row><row data-key="p1762062476073899999981312"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,832</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$168</span></div></bd></row><row data-key="p1762062476073899999981313"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 15, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,752</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$248</span></div></bd></row><row data-key="p1762062476073899999981314"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 16, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,728</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$177</span></div></bd></row><row data-key="p1762062476073899999981315"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 17, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$265</span></div></bd></row><row data-key="p1762062476073899999981316"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 22, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:43pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,737</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$177</span></div></bd></row><row data-key="p1762062476073899999981317"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 24, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:58pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,985</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$8</span></div></bd></row><row data-key="p1762062476073899999981318"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:17am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,727</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$116</span></div></bd></row><row data-key="p1762062476073899999981319"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 2, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,730</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$270</span></div></bd></row><row data-key="p1762062476073899999981320"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 3, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,760</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$240</span></div></bd></row><row data-key="p1762062476073899999981321"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 6, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:44am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,858</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$22</span></div></bd></row><row data-key="p1762062476073899999981322"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 8, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,745</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$255</span></div></bd></row><row data-key="p1762062476073899999981323"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 10, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:00am → 1:28pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$943</span></div></bd></row><row data-key="p1762062476073899999981324"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:57am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,857</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$143</span></div></bd></row><row data-key="p1762062476073899999981325"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 14, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,780</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$171</span></div></bd></row><row data-key="p1762062476073899999981326"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 15, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:25pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,808</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$175</span></div></bd></row><row data-key="p1762062476073899999981327"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 16, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:42am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,740</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$260</span></div></bd></row><row data-key="p1762062476073899999981328"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 20, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,665</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$335</span></div></bd></row><row data-key="p1762062476073899999981329"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:13pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,985</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$15</span></div></bd></row><row data-key="p1762062476073899999981330"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 22, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,290</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$935</span></div></bd></row><row data-key="p1762062476073899999981331"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 23, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 2:30pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,822</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$822</span></div></bd></row><row data-key="p1762062476073899999981332"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 24, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,685</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$315</span></div></bd></row><row data-key="p1762062476073899999981333"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 29, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:11pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,922</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$78</span></div></bd></row><row data-key="p1762062476073899999981334"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 31, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,837</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$163</span></div></bd></row><row data-key="p1762062476073899999981335"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 3, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,882</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$118</span></div></bd></row><row data-key="p1762062476073899999981336"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 6, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:22am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,878</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$122</span></div></bd></row><row data-key="p1762062476073899999981337"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 10, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,728</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$272</span></div></bd></row><row data-key="p17620624760731338"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 11, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:12pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,814</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$94</span></div></bd></row><row data-key="p17620624760731339"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,833</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$167</span></div></bd></row><row data-key="p17620624760731340"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 17, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:17pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,975</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$25</span></div></bd></row><row data-key="p17620624760731341"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 19, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,708</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$292</span></div></bd></row><row data-key="p17620624760731342"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 20, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$62</span></div></bd></row><row data-key="p17620624760731343"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:01pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,964</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$26</span></div></bd></row><row data-key="p17620624760731344"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 24, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,730</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$258</span></div></bd></row><row data-key="p17620624760731345"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 26, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:02pm → 1:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$250</span></div></bd></row><row data-key="p17620624760731346"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 27, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$157</span></div></bd></row><row data-key="p17620624760731347"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:59am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,952</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$48</span></div></bd></row><row data-key="p17620624760731348"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 1, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:47am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,815</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$155</span></div></bd></row><row data-key="p17620624760731349"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 5, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,953</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$47</span></div></bd></row><row data-key="p17620624760731350"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 8, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:49pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,985</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$15</span></div></bd></row><row data-key="p17620624760731351"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 9, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:56pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,958</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$35</span></div></bd></row><row data-key="p17620624760731352"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 11, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:44pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,990</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$10</span></div></bd></row><row data-key="p17620624760731353"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 15, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:46pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,960</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$40</span></div></bd></row><row data-key="p17620624760731354"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 17, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:18am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,928</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$72</span></div></bd></row><row data-key="p17620624760731355"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 18, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,894</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$106</span></div></bd></row><row data-key="p17620624760731356"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 19, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,672</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$524</span></div></bd></row><row data-key="p17620624760731357"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 23, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:46am → 1:31pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,852</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$777</span></div></bd></row><row data-key="p17620624760731358"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 25, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:01pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,980</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$20</span></div></bd></row><row data-key="p17620624760731359"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 26, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,929</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$71</span></div></bd></row><row data-key="p17620624760731360"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 1, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:42am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,898</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$102</span></div></bd></row><row data-key="p17620624760731361"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 2, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:47am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$90</span></div></bd></row><row data-key="p17620624760731362"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 5, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:15pm → 2:33pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,942</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$282</span></div></bd></row><row data-key="p17620624760731363"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 6, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,845</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$155</span></div></bd></row><row data-key="p17620624760731364"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 8, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p17620624760731365"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 9, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 11:32am</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,887</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$587</span></div></bd></row><row data-key="p17620624760731366"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 12, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,747</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$253</span></div></bd></row><row data-key="p17620624760731367"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:24am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,947</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$53</span></div></bd></row><row data-key="p17620624760731368"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 14, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,575</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$113</span></div></bd></row><row data-key="p17620624760731369"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 15, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:49am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,970</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$30</span></div></bd></row><row data-key="p17620624760731370"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 20, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:22pm → 1:23pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$9</span></div></bd></row><row data-key="p17620624760731371"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:20pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,990</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val zero pnl">--</span></div></bd></row><row data-key="p17620624760731372"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 22, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:31pm → 2:32pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$9</span></div></bd></row><row data-key="p17620624760731373"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 23, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,880</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$110</span></div></bd></row><row data-key="p17620624760731374"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 27, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$157</span></div></bd></row><row data-key="p17620624760731375"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,808</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$178</span></div></bd></row><row data-key="p17620624760731376"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 29, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:18am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p17620624760731377"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 30, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:54pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,968</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$22</span></div></bd></row><row data-key="p17620624760731378"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 5, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p17620624760731379"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 7, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:38am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$5</span></div></bd></row><row data-key="p17620624760731380"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 11, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:53am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,947</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$53</span></div></bd></row><row data-key="p17620624760731381"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 12, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 11:57am</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$477</span></div></bd></row><row data-key="p17620624760731382"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:03pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p17620624760731383"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 17, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:50am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,852</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$148</span></div></bd></row><row data-key="p17620624760731384"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 18, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,960</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$40</span></div></bd></row><row data-key="p17620624760731385"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 19, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:04pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,903</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$67</span></div></bd></row><row data-key="p17620624760731386"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:41pm → 3:10pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,974</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$224</span></div></bd></row><row data-key="p17620624760731387"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 24, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:16am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,907</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$86</span></div></bd></row><row data-key="p17620624760731388"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 25, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,907</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$93</span></div></bd></row><row data-key="p17620624760731389"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 26, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:37pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,555</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$400</span></div></bd></row><row data-key="p17620624760731390"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:12am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$95</span></div></bd></row><row data-key="p17620624760731391"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 3, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:19am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,863</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$137</span></div></bd></row><row data-key="p17620624760731392"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 4, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:46am → 2:04pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$210</span></div></bd></row><row data-key="p17620624760731393"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 7, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,838</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$162</span></div></bd></row><row data-key="p17620624760731394"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 8, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:26pm → 3:27pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$10</span></div></bd></row><row data-key="p17620624760731395"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 11, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p17620624760731396"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 14, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,862</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$138</span></div></bd></row><row data-key="p17620624760731397"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 18, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:01am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,833</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$167</span></div></bd></row><row data-key="p17620624760731398"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:44pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,994</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$6</span></div></bd></row><row data-key="p17620624760731399"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 23, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:43am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,847</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$153</span></div></bd></row><row data-key="p17620624760731400"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 25, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:01pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,985</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$8</span></div></bd></row><row data-key="p17620624760731401"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:48pm → 3:49pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$14</span></div></bd></row><row data-key="p17620624760731402"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 29, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,952</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$48</span></div></bd></row><row data-key="p17620624760731403"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 30, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:47pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,979</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$21</span></div></bd></row><row data-key="p17620624760731404"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 31, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,768</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p17620624760731405"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 5, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:07am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,868</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$118</span></div></bd></row><row data-key="p17620624760731406"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 7, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:43am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$212</span></div></bd></row><row data-key="p17620624760731407"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 8, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,904</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$44</span></div></bd></row><row data-key="p17620624760731408"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 12, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:21pm → 2:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,952</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$399</span></div></bd></row><row data-key="p17620624760731409"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:58am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,915</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$85</span></div></bd></row><row data-key="p17620624760731410"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 14, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,899</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$101</span></div></bd></row><row data-key="p17620624760731411"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 18, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,762</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$238</span></div></bd></row><row data-key="p17620624760731412"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 22, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,792</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$567</span></div></bd></row><row data-key="p17620624760731413"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 25, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:17am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,832</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$168</span></div></bd></row><row data-key="p17620624760731414"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,802</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$198</span></div></bd></row><row data-key="p17620624760731415"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 2, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:22am → 12:10pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$760</span></div></bd></row><row data-key="p17620624760731416"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 4, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,877</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$123</span></div></bd></row><row data-key="p17620624760731417"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 5, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:41pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,974</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$16</span></div></bd></row><row data-key="p17620624760731418"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 6, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,812</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$188</span></div></bd></row><row data-key="p17620624760731419"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 9, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:08pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p17620624760731420"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 10, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,865</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p17620624760731421"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 12, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:22pm → 1:06pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,915</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$415</span></div></bd></row><row data-key="p17620624760731422"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 16, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,888</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$112</span></div></bd></row><row data-key="p17620624760731423"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 17, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,868</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$132</span></div></bd></row><row data-key="p17620624760731424"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 19, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:03pm → 2:37pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,800</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$920</span></div></bd></row><row data-key="p17620624760731425"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 23, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:06am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,892</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$108</span></div></bd></row><row data-key="p17620624760731426"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 24, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,675</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$318</span></div></bd></row><row data-key="p17620624760731427"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 27, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:17am → 2:06pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,220</span></div></bd></row><row data-key="p17620624760731428"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 30, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:31pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,937</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$63</span></div></bd></row><row data-key="p17620624760731429"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 31, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:53am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,810</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$190</span></div></bd></row><row data-key="p17620624760731430"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 1, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,685</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$315</span></div></bd></row><row data-key="p17620624760731431"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 2, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,894</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$106</span></div></bd></row><row data-key="p17620624760731432"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 3, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:18pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,975</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$25</span></div></bd></row><row data-key="p17620624760731433"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 7, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p17620624760731434"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 10, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:06am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,860</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$140</span></div></bd></row><row data-key="p17620624760731435"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,828</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$172</span></div></bd></row><row data-key="p17620624760731436"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 14, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$45</span></div></bd></row><row data-key="p17620624760731437"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 15, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:51am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,867</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$1</span></div></bd></row><row data-key="p17620624760731438"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 17, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:15pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,945</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$36</span></div></bd></row><row data-key="p17620624760731439"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 20, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$70</span></div></bd></row><row data-key="p17620624760731440"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:40pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,905</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$95</span></div></bd></row><row data-key="p17620624760731441"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 27, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:57am → 3:55pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,935</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$58</span></div></bd></row><row data-key="p17620624760731442"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$190</span></div></bd></row><row data-key="p17620624760731443"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 1, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:12am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p17620624760731444"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 5, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,922</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$78</span></div></bd></row><row data-key="p17620624760731445"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 7, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,865</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p17620624760731446"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 11, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$90</span></div></bd></row><row data-key="p17620624760731447"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 12, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$105</span></div></bd></row><row data-key="p17620624760731448"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 13, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,522</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$478</span></div></bd></row><row data-key="p17620624760731449"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 15, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,823</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$155</span></div></bd></row><row data-key="p17620624760731450"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 18, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:18am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,903</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$97</span></div></bd></row><row data-key="p17620624760731451"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 19, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$60</span></div></bd></row><row data-key="p17620624760731452"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 20, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 2:43pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,900</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$478</span></div></bd></row><row data-key="p17620624760731453"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 21, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:47pm → 3:48pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$10</span></div></bd></row><row data-key="p17620624760731454"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 26, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p17620624760731455"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 27, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:07pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,905</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$23</span></div></bd></row><row data-key="p17620624760731456"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 28, 2023</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,737</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$191</span></div></bd></row><row data-key="p17620624760731457"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:46am → 3:04pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,900</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$472</span></div></bd></row><row data-key="p17620624760731458"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 3, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:28pm → 3:09pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,965</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$165</span></div></bd></row><row data-key="p17620624760731459"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:50am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,925</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$117</span></div></bd></row><row data-key="p17620624760731460"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,903</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$97</span></div></bd></row><row data-key="p17620624760731461"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:17am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,888</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$112</span></div></bd></row><row data-key="p17620624760731462"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:57pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$95</span></div></bd></row><row data-key="p17620624760731463"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 2:20pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$493</span></div></bd></row><row data-key="p17620624760731464"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 17, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,688</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$275</span></div></bd></row><row data-key="p17620624760731465"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,770</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$230</span></div></bd></row><row data-key="p17620624760731466"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 19, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:56am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,747</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$253</span></div></bd></row><row data-key="p17620624760731467"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 23, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:47pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,938</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$52</span></div></bd></row><row data-key="p17620624760731468"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 24, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 3:20pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,845</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$747</span></div></bd></row><row data-key="p17620624760731469"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 26, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,732</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$184</span></div></bd></row><row data-key="p17620624760731470"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 29, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:25pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,854</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$146</span></div></bd></row><row data-key="p17620624760731471"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 30, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:49am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,863</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$127</span></div></bd></row><row data-key="p17620624760731472"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 1, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$212</span></div></bd></row><row data-key="p17620624760731473"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,828</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$172</span></div></bd></row><row data-key="p17620624760731474"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 7, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p17620624760731475"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:03pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,945</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$23</span></div></bd></row><row data-key="p17620624760731476"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,747</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$243</span></div></bd></row><row data-key="p17620624760731477"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,760</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$188</span></div></bd></row><row data-key="p17620624760731478"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 13, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:10am → 2:13pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$775</span></div></bd></row><row data-key="p17620624760731479"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 14, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:27pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,990</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$10</span></div></bd></row><row data-key="p17620624760731480"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 15, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:46pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,837</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$163</span></div></bd></row><row data-key="p17620624760731481"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$78</span></div></bd></row><row data-key="p17620624760731482"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 21, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:13am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$198</span></div></bd></row><row data-key="p17620624760731483"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,867</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$133</span></div></bd></row><row data-key="p17620624760731484"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 27, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:12pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,995</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$5</span></div></bd></row><row data-key="p17620624760731485"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 28, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,810</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$180</span></div></bd></row><row data-key="p17620624760731486"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 29, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:49pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$10</span></div></bd></row><row data-key="p17620624760731487"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 1, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,820</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$180</span></div></bd></row><row data-key="p17620624760731488"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,762</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$189</span></div></bd></row><row data-key="p17620624760731489"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 6, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:03am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,898</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$88</span></div></bd></row><row data-key="p17620624760731490"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 7, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,805</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$195</span></div></bd></row><row data-key="p17620624760731491"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 11, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:36pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,870</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$130</span></div></bd></row><row data-key="p17620624760731492"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$60</span></div></bd></row><row data-key="p17620624760731493"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 13, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:09pm → 3:29pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,915</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$570</span></div></bd></row><row data-key="p17620624760731494"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 15, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,782</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$193</span></div></bd></row><row data-key="p17620624760731495"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 19, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,697</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$303</span></div></bd></row><row data-key="p17620624760731496"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 20, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,410</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$590</span></div></bd></row><row data-key="p17620624760731497"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 21, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:28pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,868</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$80</span></div></bd></row><row data-key="p17620624760731498"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:54pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$46</span></div></bd></row><row data-key="p17620624760731499"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 25, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:41pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,980</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$6</span></div></bd></row><row data-key="p17620624760731500"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 27, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:40pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,998</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$8</span></div></bd></row><row data-key="p17620624760731501"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 28, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,772</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$186</span></div></bd></row><row data-key="p17620624760731502"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:09pm → 3:10pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$9</span></div></bd></row><row data-key="p17620624760731503"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 3, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$60</span></div></bd></row><row data-key="p17620624760731504"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:10pm → 2:01pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,890</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$560</span></div></bd></row><row data-key="p17620624760731505"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 5, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,763</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$237</span></div></bd></row><row data-key="p17620624760731506"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,845</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$141</span></div></bd></row><row data-key="p17620624760731507"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,703</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$277</span></div></bd></row><row data-key="p17620624760731508"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 11, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:05pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,943</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$57</span></div></bd></row><row data-key="p17620624760731509"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:55am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,645</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$335</span></div></bd></row><row data-key="p17620624760731510"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,737</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$176</span></div></bd></row><row data-key="p17620624760731511"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:18pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,820</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$180</span></div></bd></row><row data-key="p17620624760731512"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 23, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,872</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$128</span></div></bd></row><row data-key="p17620624760731513"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 25, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:05pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,867</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$133</span></div></bd></row><row data-key="p17620624760731514"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 26, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:02am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,884</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$116</span></div></bd></row><row data-key="p17620624760731515"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 1, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:01pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,420</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$566</span></div></bd></row><row data-key="p17620624760731516"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:32pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,970</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$30</span></div></bd></row><row data-key="p17620624760731517"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 6, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,849</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$151</span></div></bd></row><row data-key="p17620624760731518"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 7, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:56am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,805</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$163</span></div></bd></row><row data-key="p17620624760731519"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:42am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,863</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$137</span></div></bd></row><row data-key="p17620624760731520"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,837</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$163</span></div></bd></row><row data-key="p17620624760731521"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 14, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:33pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,970</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$30</span></div></bd></row><row data-key="p17620624760731522"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 15, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,847</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$153</span></div></bd></row><row data-key="p17620624760731523"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:43am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,839</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$261</span></div></bd></row><row data-key="p17620624760731524"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 20, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:43am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,935</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$28</span></div></bd></row><row data-key="p17620624760731525"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 21, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$105</span></div></bd></row><row data-key="p17620624760731526"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:07am → 2:03pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,818</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,063</span></div></bd></row><row data-key="p17620624760731527"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 24, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:55am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,875</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p17620624760731528"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 29, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,797</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$189</span></div></bd></row><row data-key="p17620624760731529"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 30, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:43pm → 3:25pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$438</span></div></bd></row><row data-key="p17620624760731530"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:16pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,942</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$48</span></div></bd></row><row data-key="p17620624760731531"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 5, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,783</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$217</span></div></bd></row><row data-key="p17620624760731532"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 7, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$178</span></div></bd></row><row data-key="p17620624760731533"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,748</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$252</span></div></bd></row><row data-key="p17620624760731534"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 11, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:00am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,920</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$80</span></div></bd></row><row data-key="p17620624760731535"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,750</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$167</span></div></bd></row><row data-key="p17620624760731536"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 14, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:53pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$80</span></div></bd></row><row data-key="p17620624760731537"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 17, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:58am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,804</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$196</span></div></bd></row><row data-key="p17620624760731538"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:39pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,934</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$52</span></div></bd></row><row data-key="p17620624760731539"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 21, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:02am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p17620624760731540"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 25, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,742</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$248</span></div></bd></row><row data-key="p17620624760731541"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 26, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:51am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,860</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$140</span></div></bd></row><row data-key="p17620624760731542"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,823</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$177</span></div></bd></row><row data-key="p17620624760731543"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 5, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:50am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,814</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$186</span></div></bd></row><row data-key="p17620624760731544"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:04pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,905</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$36</span></div></bd></row><row data-key="p17620624760731545"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,828</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$172</span></div></bd></row><row data-key="p17620624760731546"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,879</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$107</span></div></bd></row><row data-key="p17620624760731547"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 15, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,738</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$425</span></div></bd></row><row data-key="p1762062476073100000021548"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:52am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,795</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$195</span></div></bd></row><row data-key="p1762062476073100000021549"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:07am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,857</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$143</span></div></bd></row><row data-key="p1762062476073100000021550"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 23, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:16am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$38</span></div></bd></row><row data-key="p1762062476073100000021551"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 25, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$155</span></div></bd></row><row data-key="p1762062476073100000021552"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 26, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:59am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,822</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$164</span></div></bd></row><row data-key="p1762062476073100000021553"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 31, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,590</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$388</span></div></bd></row><row data-key="p1762062476073100000021554"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 5, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:25am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,918</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$69</span></div></bd></row><row data-key="p1762062476073100000021555"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 6, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,825</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$175</span></div></bd></row><row data-key="p1762062476073100000021556"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,815</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$185</span></div></bd></row><row data-key="p1762062476073100000021557"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:56am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,645</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$355</span></div></bd></row><row data-key="p1762062476073100000021558"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,847</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$106</span></div></bd></row><row data-key="p1762062476073100000021559"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 13, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:51am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,710</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$290</span></div></bd></row><row data-key="p1762062476073100000021560"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 14, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,827</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$173</span></div></bd></row><row data-key="p1762062476073100000021561"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 15, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$265</span></div></bd></row><row data-key="p1762062476073100000021562"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:11pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,869</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$121</span></div></bd></row><row data-key="p1762062476073100000021563"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 19, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,780</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$220</span></div></bd></row><row data-key="p1762062476073100000021564"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 21, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,777</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$204</span></div></bd></row><row data-key="p1762062476073100000021565"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 27, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$105</span></div></bd></row><row data-key="p1762062476073100000021566"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 29, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,807</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$622</span></div></bd></row><row data-key="p1762062476073100000021567"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 30, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:51pm → 3:52pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,995</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$5</span></div></bd></row><row data-key="p1762062476073100000021568"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,758</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$195</span></div></bd></row><row data-key="p1762062476073100000021569"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:04pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,825</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476073100000021570"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:33pm → 3:40pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,998</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$17</span></div></bd></row><row data-key="p1762062476073100000021571"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 11, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:12pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,987</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$13</span></div></bd></row><row data-key="p1762062476073100000021572"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,760</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$240</span></div></bd></row><row data-key="p1762062476073100000021573"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 13, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:45am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,820</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$170</span></div></bd></row><row data-key="p1762062476073100000021574"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 17, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:46am → 12:48pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,008</span></div></bd></row><row data-key="p1762062476073100000021575"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:31pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,365</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$120</span></div></bd></row><row data-key="p1762062476073100000021576"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 19, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,807</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$178</span></div></bd></row><row data-key="p1762062476073100000021577"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 20, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:39pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,993</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$8</span></div></bd></row><row data-key="p1762062476073100000021578"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 24, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:54am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$60</span></div></bd></row><row data-key="p1762062476073100000021579"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 30, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:05am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,740</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$220</span></div></bd></row><row data-key="p1762062476073100000021580"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,827</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$173</span></div></bd></row><row data-key="p1762062476073100000021581"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:35pm → 3:36pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$10</span></div></bd></row><row data-key="p1762062476073100000021582"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:43am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p1762062476073100000021583"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 9, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:42am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,760</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$240</span></div></bd></row><row data-key="p1762062476073100000021584"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:56am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$135</span></div></bd></row><row data-key="p1762062476073100000021585"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 11, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:10pm → 3:11pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$2,000</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$10</span></div></bd></row><row data-key="p1762062476073100000021586"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 14, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,918</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$82</span></div></bd></row><row data-key="p1762062476073100000021587"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:55pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,935</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$65</span></div></bd></row><row data-key="p1762062476073100000021588"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:43pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,932</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$68</span></div></bd></row><row data-key="p1762062476073100000021589"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:04am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,820</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$180</span></div></bd></row><row data-key="p1762062476073100000021590"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 23, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 12:28pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,803</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$903</span></div></bd></row><row data-key="p1762062476073100000021591"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 28, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,672</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$233</span></div></bd></row><row data-key="p1762062476073100000021592"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 29, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:01pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,973</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$27</span></div></bd></row><row data-key="p1762062476073100000021593"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 30, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:32am → 1:41pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,965</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$178</span></div></bd></row><row data-key="p1762062476073100000021594"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 1, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$22</span></div></bd></row><row data-key="p1762062476073100000021595"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 5, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,875</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$125</span></div></bd></row><row data-key="p1762062476073100000021596"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 6, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,880</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$120</span></div></bd></row><row data-key="p1762062476073100000021597"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 7, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,570</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$430</span></div></bd></row><row data-key="p1762062476073100000021598"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 8, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,825</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476073100000021599"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 13, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:28am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,938</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$62</span></div></bd></row><row data-key="p1762062476073100000021600"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,840</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$160</span></div></bd></row><row data-key="p1762062476073100000021601"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 19, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,692</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$308</span></div></bd></row><row data-key="p1762062476073100000021602"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 21, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:44pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,979</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$21</span></div></bd></row><row data-key="p1762062476073100000021603"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 22, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:12pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,944</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$56</span></div></bd></row><row data-key="p1762062476073100000021604"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Nov 26, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,875</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$125</span></div></bd></row><row data-key="p1762062476073100000021605"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 2, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:42am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,870</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$130</span></div></bd></row><row data-key="p1762062476073100000021606"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 3, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:48pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,995</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$5</span></div></bd></row><row data-key="p1762062476073100000021607"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 4, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$212</span></div></bd></row><row data-key="p1762062476073100000021608"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 5, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:47pm → 1:24pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,940</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$325</span></div></bd></row><row data-key="p1762062476073100000021609"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 10, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 3:06pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,863</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$650</span></div></bd></row><row data-key="p1762062476073100000021610"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 11, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:27am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$45</span></div></bd></row><row data-key="p1762062476073100000021611"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 12, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,914</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$29</span></div></bd></row><row data-key="p1762062476073100000021612"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 16, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,912</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$88</span></div></bd></row><row data-key="p1762062476073100000021613"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 17, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:10am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,925</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$56</span></div></bd></row><row data-key="p1762062476073100000021614"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 18, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:52pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,608</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,613</span></div></bd></row><row data-key="p1762062476073200000051615"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 20, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,924</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$76</span></div></bd></row><row data-key="p1762062476074200000051616"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 23, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:47am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,929</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$71</span></div></bd></row><row data-key="p1762062476074200000051617"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 26, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,812</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$188</span></div></bd></row><row data-key="p1762062476074200000051618"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Dec 30, 2024</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:02pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$45</span></div></bd></row><row data-key="p1762062476074200000051619"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 3, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:04am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,828</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$172</span></div></bd></row><row data-key="p1762062476074200000051620"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 6, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:54am → 1:46pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,919</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$442</span></div></bd></row><row data-key="p1762062476074200000051621"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 8, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,780</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$220</span></div></bd></row><row data-key="p1762062476074200000051622"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 13, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:45am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,775</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$225</span></div></bd></row><row data-key="p1762062476074200000051623"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 15, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:29pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,990</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$10</span></div></bd></row><row data-key="p1762062476074200000051624"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 16, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:09am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,840</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$150</span></div></bd></row><row data-key="p1762062476074200000051625"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 17, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:54am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,798</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$202</span></div></bd></row><row data-key="p1762062476074200000051626"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 21, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:29pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,958</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$42</span></div></bd></row><row data-key="p1762062476074200000051627"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 22, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,783</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$207</span></div></bd></row><row data-key="p1762062476074200000051628"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 23, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,797</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$203</span></div></bd></row><row data-key="p1762062476074200000051629"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 27, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,908</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$82</span></div></bd></row><row data-key="p1762062476074200000051630"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 28, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:58am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,960</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$40</span></div></bd></row><row data-key="p1762062476074200000051631"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 30, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:03pm → 3:39pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,995</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$30</span></div></bd></row><row data-key="p1762062476074200000051632"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jan 31, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 2:07pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,752</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,182</span></div></bd></row><row data-key="p1762062476074200000051633"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 3, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p1762062476074200000051634"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 4, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,853</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$147</span></div></bd></row><row data-key="p1762062476074200000051635"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 5, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:10am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476074200000051636"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 10, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:52pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,840</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$153</span></div></bd></row><row data-key="p1762062476074200000051637"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,755</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$245</span></div></bd></row><row data-key="p1762062476074200000051638"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 12, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p1762062476074200000051639"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 13, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,848</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$152</span></div></bd></row><row data-key="p1762062476074200000051640"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 19, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,668</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$332</span></div></bd></row><row data-key="p1762062476074200000051641"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 26, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:02am → 12:57pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,925</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$375</span></div></bd></row><row data-key="p1762062476074200000051642"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Feb 28, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$157</span></div></bd></row><row data-key="p1762062476074200000051643"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 4, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:56pm → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,970</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$15</span></div></bd></row><row data-key="p1762062476074200000051644"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 5, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:52pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,745</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$255</span></div></bd></row><row data-key="p1762062476074200000051645"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 6, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,680</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$500</span></div></bd></row><row data-key="p1762062476074200000051646"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 7, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:03pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,873</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$127</span></div></bd></row><row data-key="p1762062476074200000051647"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:53pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,953</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$37</span></div></bd></row><row data-key="p1762062476074200000051648"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 14, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:00am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,795</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$205</span></div></bd></row><row data-key="p1762062476074200000051649"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 17, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:37pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,950</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$50</span></div></bd></row><row data-key="p1762062476074200000051650"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 19, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:50am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,690</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$310</span></div></bd></row><row data-key="p1762062476074200000051651"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 20, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,928</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$72</span></div></bd></row><row data-key="p1762062476074200000051652"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 21, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:57am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,680</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$320</span></div></bd></row><row data-key="p1762062476074200000051653"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 24, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:49am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,905</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$95</span></div></bd></row><row data-key="p1762062476074200000051654"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 25, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,773</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$215</span></div></bd></row><row data-key="p1762062476074200000051655"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 27, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,840</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$160</span></div></bd></row><row data-key="p1762062476074200000051656"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Mar 31, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,795</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$205</span></div></bd></row><row data-key="p1762062476074200000051657"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 1, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:52am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,828</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$172</span></div></bd></row><row data-key="p1762062476074200000051658"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 2, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,813</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$187</span></div></bd></row><row data-key="p1762062476074200000051659"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 9, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:03pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,755</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$245</span></div></bd></row><row data-key="p1762062476074200000051660"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:32pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,775</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$225</span></div></bd></row><row data-key="p1762062476074200000051661"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 16, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:25am → 1:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,735</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,205</span></div></bd></row><row data-key="p1762062476074200000051662"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 17, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:20pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,730</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$200</span></div></bd></row><row data-key="p1762062476074200000051663"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 22, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,770</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$223</span></div></bd></row><row data-key="p1762062476074200000051664"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 24, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,800</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$200</span></div></bd></row><row data-key="p1762062476074200000051665"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 25, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:24pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,875</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$125</span></div></bd></row><row data-key="p1762062476074200000051666"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 29, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:56am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,793</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$207</span></div></bd></row><row data-key="p1762062476074200000051667"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Apr 30, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:08am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,835</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476074200000051668"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 1, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:38am → 3:51pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,705</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$278</span></div></bd></row><row data-key="p1762062476074200000051669"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 2, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:59am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,868</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$112</span></div></bd></row><row data-key="p1762062476074200000051670"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 5, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,755</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$230</span></div></bd></row><row data-key="p1762062476074200000051671"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 6, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,795</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$205</span></div></bd></row><row data-key="p1762062476074200000051672"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 7, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,560</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$245</span></div></bd></row><row data-key="p1762062476074200000051673"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 8, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:24am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,710</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$283</span></div></bd></row><row data-key="p1762062476074200000051674"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 12, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:05pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,925</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$65</span></div></bd></row><row data-key="p1762062476074200000051675"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 13, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:27am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,943</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$57</span></div></bd></row><row data-key="p1762062476074200000051676"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 15, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:05am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,763</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$237</span></div></bd></row><row data-key="p1762062476074200000051677"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 16, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:07pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,788</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$212</span></div></bd></row><row data-key="p1762062476074200000051678"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 19, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:41am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$45</span></div></bd></row><row data-key="p1762062476074200000051679"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 20, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,723</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$133</span></div></bd></row><row data-key="p1762062476074200000051680"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 21, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:09am → 1:21pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,843</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$848</span></div></bd></row><row data-key="p1762062476074200000051681"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 22, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:16pm → 3:56pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,863</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$127</span></div></bd></row><row data-key="p1762062476074200000051682"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 23, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:48am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,645</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$348</span></div></bd></row><row data-key="p1762062476074200000051683"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 27, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:47am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,943</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$57</span></div></bd></row><row data-key="p1762062476074200000051684"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">May 30, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,745</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$198</span></div></bd></row><row data-key="p1762062476074200000051685"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 2, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:52am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,928</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$72</span></div></bd></row><row data-key="p1762062476074200000051686"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 3, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,790</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$210</span></div></bd></row><row data-key="p1762062476074200000051687"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 5, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:18am → 12:53pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,927</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$357</span></div></bd></row><row data-key="p1762062476074200000051688"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 9, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,740</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$260</span></div></bd></row><row data-key="p1762062476074200000051689"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 10, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:44am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,753</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$237</span></div></bd></row><row data-key="p1762062476074200000051690"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:56am → 2:03pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,873</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$583</span></div></bd></row><row data-key="p1762062476074200000051691"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 12, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,860</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$140</span></div></bd></row><row data-key="p1762062476074200000051692"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 13, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:23am → 2:34pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,880</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$550</span></div></bd></row><row data-key="p1762062476074200000051693"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 16, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,945</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$55</span></div></bd></row><row data-key="p1762062476074200000051694"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 18, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,730</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$128</span></div></bd></row><row data-key="p1762062476074200000051695"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 23, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:31pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$45</span></div></bd></row><row data-key="p1762062476074200000051696"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 24, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:03am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,840</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$160</span></div></bd></row><row data-key="p1762062476074200000051697"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 26, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,850</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$150</span></div></bd></row><row data-key="p1762062476074200000051698"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 27, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:45am → 2:29pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,915</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$420</span></div></bd></row><row data-key="p1762062476074200000051699"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jun 30, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">3:06pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,780</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$205</span></div></bd></row><row data-key="p1762062476074200000051700"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 1, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:15pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,878</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$122</span></div></bd></row><row data-key="p1762062476074200000051701"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 2, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,898</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$102</span></div></bd></row><row data-key="p1762062476074200000051702"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 8, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:26pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,892</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$98</span></div></bd></row><row data-key="p1762062476074200000051703"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 10, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 3:53pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,808</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$182</span></div></bd></row><row data-key="p1762062476074200000051704"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,865</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$125</span></div></bd></row><row data-key="p1762062476074200000051705"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 14, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:03am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,885</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$115</span></div></bd></row><row data-key="p1762062476074200000051706"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 16, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:56pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,988</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$12</span></div></bd></row><row data-key="p1762062476074200000051707"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 17, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:06am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,895</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$105</span></div></bd></row><row data-key="p1762062476074200000051708"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 21, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,965</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$25</span></div></bd></row><row data-key="p1762062476074200000051709"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 22, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:41pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,995</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$5</span></div></bd></row><row data-key="p1762062476074200000051710"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 23, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:55am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,910</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$90</span></div></bd></row><row data-key="p1762062476074299999951711"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 24, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:51am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,805</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$165</span></div></bd></row><row data-key="p1762062476074299999951712"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 25, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:35am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,790</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$210</span></div></bd></row><row data-key="p1762062476074299999951713"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Jul 30, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:40pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,660</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,560</span></div></bd></row><row data-key="p1762062476074299999951714"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 4, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:50am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,935</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$65</span></div></bd></row><row data-key="p1762062476074299999951715"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 6, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:55am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,818</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$182</span></div></bd></row><row data-key="p1762062476074299999951716"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 8, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,898</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$92</span></div></bd></row><row data-key="p1762062476074299999951717"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:07pm → 2:45pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,837</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$744</span></div></bd></row><row data-key="p1762062476074299999951718"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 12, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:52am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,900</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$100</span></div></bd></row><row data-key="p1762062476074299999951719"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 14, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,853</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$147</span></div></bd></row><row data-key="p1762062476074299999951720"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 22, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:45am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,967</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$33</span></div></bd></row><row data-key="p1762062476074299999951721"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 25, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:13am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,865</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$63</span></div></bd></row><row data-key="p1762062476074299999951722"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 26, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:17pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,947</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$53</span></div></bd></row><row data-key="p1762062476074299999951723"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 27, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:34am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,922</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$78</span></div></bd></row><row data-key="p1762062476074299999951724"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Aug 28, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:29pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,960</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$40</span></div></bd></row><row data-key="p1762062476074299999951725"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 3, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:41am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,818</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$162</span></div></bd></row><row data-key="p1762062476074299999951726"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 4, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,793</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$207</span></div></bd></row><row data-key="p1762062476074299999951727"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 8, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:39am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,830</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$150</span></div></bd></row><row data-key="p1762062476074299999951728"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 9, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">1:26pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,925</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$75</span></div></bd></row><row data-key="p1762062476074299999951729"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 11, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:33am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,858</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$142</span></div></bd></row><row data-key="p1762062476074299999951730"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 12, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:55am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,830</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$120</span></div></bd></row><row data-key="p1762062476074299999951731"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 15, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:45am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$97</span></div></bd></row><row data-key="p1762062476074299999951732"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 17, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:01pm → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,420</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$400</span></div></bd></row><row data-key="p1762062476074299999951733"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 18, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$60</span></div></bd></row><row data-key="p1762062476074299999951734"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 19, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:49pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,923</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$77</span></div></bd></row><row data-key="p1762062476074299999951735"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 22, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,930</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$70</span></div></bd></row><row data-key="p1762062476074299999951736"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 25, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:43am → 1:58pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,937</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$332</span></div></bd></row><row data-key="p1762062476074299999951737"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Sep 30, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:22pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,855</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$145</span></div></bd></row><row data-key="p1762062476074299999951738"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 1, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:36am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,880</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$120</span></div></bd></row><row data-key="p1762062476074299999951739"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 3, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:32am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,810</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$63</span></div></bd></row><row data-key="p1762062476074299999951740"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 6, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:46am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,925</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$75</span></div></bd></row><row data-key="p1762062476074299999951741"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 8, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:31am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p1762062476074299999951742"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 13, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:10pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,860</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$140</span></div></bd></row><row data-key="p1762062476074299999951743"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 14, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:37am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,880</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$120</span></div></bd></row><row data-key="p1762062476074299999951744"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 16, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:42am → 2:01pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl lred">Stop Loss</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,710</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val neg pnl">-$1,320</span></div></bd></row><row data-key="p1762062476074299999951745"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 17, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">2:07pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,955</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$45</span></div></bd></row><row data-key="p1762062476074299999951746"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 20, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:40am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,890</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$110</span></div></bd></row><row data-key="p1762062476074299999951747"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 21, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">10:53am → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,818</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$182</span></div></bd></row><row data-key="p1762062476074299999951748"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 23, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">12:07pm → 4:00pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl">Expired</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$117</span></div></bd></row><row data-key="p1762062476074299999951749"><bd><div class="cell symbol" style="width:50%;flex:2;"><div class="clip"><span class="sym" style="margin-right:.6rem;">SPX</span><span>Short Put Spread</span></div></div><div class="sorted cell closeTime" style="width:50%;flex:2.5;"><div class="clip" style="font-size:1.4rem;line-height:1.4;margin-top:.2rem;">Oct 24, 2025</div><div class="clip" style="font-size:1.2rem;line-height:1.4;margin-top:.2rem;">11:21am → 3:50pm</div></div><div class="cell status" style="width:25%;flex:1;"><span class="lbl" title="Closed before expiration">Before Exp</span></div><div class="cell text-right nfont risk" style="width:25%;flex:1;"><span class="val pos">$1,883</span></div><div class="cell text-right nfont pnl" style="width:25%;flex:1;"><span class="val pos pnl">$107</span></div></bd></row></bd>
"""

df = ifera.parse_trade_log(trade_grid)

full_period = df.index.max() - df.index.min()
num_years = full_period.days / 365.25

dff = ifera.get_filters("SPX-ORB-L")

left_only = [
    "skip_cpi",
    "skip_eom",
    "skip_eoq",
    "skip_fm",
    "skip_fomc",
    "skip_fw",
    "skip_me",
    "skip_pay",
    "skip_pce",
    "skip_ppi",
    "skip_tw",
    "first_bo",
]
right_only = ["reward_per_risk"]

t = time.time()
X, y, splits = ifera.prepare_splits(df, dff, 20, left_only, right_only, torch.device("cuda"), torch.float32, max_splits_per_filter=10, max_depth=1, min_samples=200)
print(f"prepare_splits time: {time.time() - t:.2f} sec, number of splits: {len(splits)}")

prepare_splits time: 0.89 sec, number of splits: 52


In [7]:
import datetime as dt
from dateutil.relativedelta import relativedelta

d = dt.date(2020, 1, 1)

d + relativedelta(years=1, days=-1)


datetime.date(2020, 12, 31)